# The data
This implementation will use the various images of ALS which we collected. The dataset contains more than 500 images of 25 different representational letters. 

## Loading the data
we want python to point to the location where the images are located. This way instead of loading a whole file path, we can simply just use the name of the file.

In [56]:
# load_img allows us to load an image from a file as a PIL object
from keras.preprocessing.image import load_img 
# img_to_array allows us to convert the PIL object into a NumPy array
from keras.preprocessing.image import img_to_array 
# preproccess_input is meant to prepare your image into the format the model requires. 
# You should load images with the Keras load_img function so that you guarantee the images you load are compatible with the preprocess_input function.
from keras.applications.vgg16 import preprocess_input 

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle

path = r"C:\Users\matan\My PC (DESKTOP-RLTMVS3)\Desktop\פרויקט גמר\dataset\dark photos"
# change the working directory to the path where the images are located
os.chdir(path)

# this list holds all the image filename
flowers = []

# creates a ScandirIterator aliased as files
with os.scandir(path) as files:
  # loops through each file in the directory
    for file in files:
        if file.name.endswith('.jpg'):
          # adds only the image files to the flowers list
            flowers.append(file.name)

# The model
Using a pre-trained neural network to extract a feature vector from images and cluster the images based on how similar the feature vectors are.

The pre-trained model that will be used in this tutorial is the VGG16 convolutional neural network (CNN), which is considered to be state of the art for image recognition tasks. We are going to be using this model as a feature extractor only, meaning that we will remove the final (prediction) layer so that we can obtain a feature vector.

In [36]:
from keras.applications.vgg16 import VGG16 
from keras.models import Model

model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

## Data Preprocessing
This is where we put the load_img() and preprocess_input() methods to use. When loading the images we are going to set the target size to (224, 224) because the VGG model expects the images it receives to be 224x224 NumPy arrays.

In [37]:
def extract_features(file, model):
    # load the image as a 224x224 array
    img = load_img(file, target_size=(224,224),grayscale=True)
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img) 
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3) 
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    return features

In [57]:
data = {}

# lop through each image in the dataset
for flower in flowers:
    # try to extract the features and update the dictionary
    try:
        feat = extract_features(flower,model)
        data[flower] = feat
    # if something fails, save the extracted features as a pickle file (optional)
    except:
        print("error extracting feature from the image")
          
# get a list of the filenames
filenames = np.array(list(data.keys()))

In [58]:
# get a list of just the features
feat = np.array(list(data.values()))

# reshape so that there are 210 samples of 4096 vectors
feat = feat.reshape(-1,4096)

# get the unique labels 
unique_labels = list(map(chr, range(97, 123)))

## Dimensionality Reduction
Simply put, if you are working with data and have a lot of variables to consider (in our case 4096), PCA allows you to reduce the number of variables while preserving as much information from the original set as possible.

The number of dimensions to reduce down to is up to you and I'm sure there's a method for finding the best number of components to use, but for this case, I just chose 100 as an arbitrary number.

In [67]:
# PCA for reducing the dimensions of our feature vector
from sklearn.decomposition import PCA

pca = PCA(n_components=100, random_state=22)
pca.fit(feat)
x = pca.transform(feat)

## KMeans clustering
This algorithm will allow us to group our feature vectors into k clusters. Each cluster should contain images that are visually similar. In this case, we know there are 10 different species of flowers so we can have k = 26.

In [ ]:
# clustering and dimension reduction
from sklearn.cluster import KMeans

# cluster feature vectors
kmeans = KMeans(n_clusters=len(unique_labels), random_state=22)
kmeans.fit(x)

In [ ]:
# holds the cluster id and the images { id: [images] }
groups = {}
for file, cluster in zip(filenames,kmeans.labels_):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(file)
    else:
        groups[cluster].append(file)

In [50]:
# function that lets you view a cluster (based on identifier)        
def view_cluster(cluster):
    plt.figure(figsize = (25,25));
    # gets the list of filenames for a cluster
    files = groups[cluster]
    # only allow up to 30 images to be shown at a time
    if len(files) > 30:
        print(f"Clipping cluster size from {len(files)} to 30")
        files = files[:29]
    # plot each image in the cluster
    for index, file in enumerate(files):
        plt.subplot(10,10,index+1);
        img = load_img(file)
        img = np.array(img)
        plt.imshow(img)
        plt.axis('off')

In [68]:
import shutil

parent_dir = r"C:\Users\matan\My PC (DESKTOP-RLTMVS3)\Desktop\פרויקט גמר\dataset\dark photos"

for i in range(len(groups)):
    path = os.path.join(parent_dir, str(i))
    os.makedirs(path)
    for s in groups[i]:
        if os.path.isfile(s): 
            shutil.move(s, path)
    print(i, groups[i])

0 ['A109.jpg', 'A110.jpg', 'A111.jpg', 'A121.jpg', 'A122.jpg', 'A123.jpg', 'A85.jpg', 'A86.jpg', 'A87.jpg', 'A97.jpg', 'A98.jpg', 'A99.jpg']
1 ['X13.jpg', 'X14.jpg', 'X15.jpg', 'X16.jpg', 'X25.jpg', 'X26.jpg', 'X27.jpg', 'X28.jpg', 'X37.jpg', 'X38.jpg', 'X39.jpg', 'X40.jpg']
2 ['Q169.jpg', 'Q170.jpg', 'Q171.jpg', 'Q172.jpg', 'Q173.jpg', 'Q181.jpg', 'Q182.jpg', 'Q183.jpg', 'Q184.jpg', 'Q185.jpg']
3 ['A37.jpg', 'A38.jpg', 'A39.jpg', 'A49.jpg', 'A50.jpg', 'A51.jpg', 'A73.jpg', 'A74.jpg', 'A75.jpg']
4 ['W1.jpg', 'W13.jpg', 'W14.jpg', 'W15.jpg', 'W2.jpg', 'W25.jpg', 'W26.jpg', 'W27.jpg', 'W3.jpg']
5 ['K13.jpg', 'K14.jpg', 'K15.jpg', 'K16.jpg', 'K37.jpg', 'K38.jpg', 'K39.jpg', 'K40.jpg']
6 ['S13.jpg', 'S14.jpg', 'S15.jpg', 'S16.jpg', 'S25.jpg', 'S26.jpg', 'S27.jpg', 'S28.jpg']
7 ['B25.jpg', 'B26.jpg', 'B27.jpg', 'B37.jpg', 'B38.jpg', 'B39.jpg']
8 ['B1.jpg', 'B13.jpg', 'B14.jpg', 'B15.jpg', 'B2.jpg', 'B3.jpg']
9 ['F25.jpg', 'F26.jpg', 'F27.jpg', 'F28.jpg', 'F29.jpg', 'F30.jpg']
10 ['F37.jpg',

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\matan\\My PC (DESKTOP-RLTMVS3)\\Desktop\\פרויקט גמר\\dataset\\dark photos\\15'